In [ ]:
! nvidia-smi

Wed May 18 15:26:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 데이터 준비

In [ ]:
# 출처: https://github.com/e9t/nsmc
import pandas as pd
df_train = pd.read_csv('ratings_train.txt', sep='\t')
df_test = pd.read_csv('ratings_test.txt', sep='\t')

df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149995 entries, 0 to 149994
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49997 entries, 0 to 49996
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


## 전처리 (토큰화)

In [ ]:
!pip install tensorflow_text

import tensorflow as tf 
import tensorflow_hub as hub
import tensorflow_text as text
tf.__version__

     |████████████████████████████████| 4.6 MB 12.4 MB/s 
     |████████████████████████████████| 511.7 MB 4.2 kB/s 
     |████████████████████████████████| 438 kB 39.8 MB/s 
     |████████████████████████████████| 5.8 MB 36.2 MB/s 
     |████████████████████████████████| 1.6 MB 37.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: flatbuffers
 

'2.9.0'

In [ ]:
preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")

text_inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string)]
tokenize = hub.KerasLayer(preprocessor.tokenize)
tokenized_inputs = [tokenize(segment) for segment in text_inputs]

seq_length = 32  # 선택
bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs,
    arguments=dict(seq_length=seq_length))  
encoder_inputs = bert_pack_inputs(tokenized_inputs)

preprocessed_text = tf.keras.Model(text_inputs, encoder_inputs)
preprocessed_text.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None,)]                 0         
                                                                 
 keras_layer (KerasLayer)    (None, None, None)        0         
                                                                 
 keras_layer_1 (KerasLayer)  {'input_word_ids': (None  0         
                             , 32),                              
                              'input_type_ids': (None            
                             , 32),                              
                              'input_mask': (None, 32            
                             )}                                  
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print(df_test['document'][2])
preprocessed_text(tf.constant([df_test['document'][2]]))

뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아


{'input_mask': <tf.Tensor: shape=(1, 32), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(1, 32), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 32), dtype=int32, numpy=
 array([[   101,   9303,  21711,   9638,   9926,  34907,  22879,    119,
            119,    119,    119,   8982, 119022,  18623,   9523,  28578,
          10150,  34907,   9715,  26344,   9074,  54141, 119168,    100,
            102,      0,      0,      0,      0,      0,      0,      0]],
       dtype=int32)>}

## 모델링

In [ ]:
model_url = "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4"

encoder = hub.KerasLayer(model_url, trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]
output = tf.keras.layers.Dense(2, activation='softmax')(pooled_output)
model = tf.keras.Model(text_inputs, output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       (None, None, None)   0           ['input_1[0][0]']                
                                                                                                  
 keras_layer_1 (KerasLayer)     {'input_word_ids':   0           ['keras_layer[0][0]']            
                                (None, 32),                                                       
                                 'input_type_ids':                                                
                                (None, 32),                                                 

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=5e-5), 
              loss='sparse_categorical_crossentropy',
              metrics='accuracy'
              )

In [ ]:
model.fit(df_train['document'], df_train['label'],  
          validation_data=(df_test['document'], df_test['label']), epochs=1, batch_size=64)

2344/2344 [==============================] - 2851s 1s/step - loss: 0.4042 - accuracy: 0.8130 - val_loss: 0.3580 - val_accuracy: 0.8407


In [ ]:
model.predict(['영화가 재미 있어요'])

1/1 [==============================] - 0s 86ms/step


array([[0.17240001, 0.8276    ]], dtype=float32)

In [ ]:
model.predict(['영화가 재미 없어요'])

1/1 [==============================] - 0s 85ms/step


array([[0.99759644, 0.00240352]], dtype=float32)

In [ ]:
def run_bert_cls():
    print('==================== BERT를 활용한 문장의 긍부정 예측 ====================')
    while True:
        sentence =input('문장을 입력해주세요: ')
        if sentence == '0':
            break
        pred = model.predict([sentence])[0]
        negative = pred[0] * 100
        positive = pred[1] * 100
        print('긍정일 확률: {:.2f}%'.format(positive))
        print('부정일 확률: {:.2f}%'.format(negative))

In [ ]:
run_bert_cls()

==================== BERT를 활용한 문장의 긍부정 예측 ====================
문장을 입력해주세요: 배우들의 연기력이 뛰어나요
1/1 [==============================] - 0s 45ms/step
긍정일 확률: 85.77%
부정일 확률: 14.23%
문장을 입력해주세요: 음악이 형편없어요
1/1 [==============================] - 0s 44ms/step
긍정일 확률: 33.71%
부정일 확률: 66.29%
문장을 입력해주세요: 스토리가 흥미진진해요
1/1 [==============================] - 0s 45ms/step
긍정일 확률: 93.47%
부정일 확률: 6.53%
문장을 입력해주세요: 0
